In [1]:
import pandas as pd

data_train = pd.read_csv('00_gt/gt.csv')
y_train = data_train.class_id
data_train.head()

,filename,class_id
0,00000.png,0
1,00001.png,0
2,00002.png,0
3,00003.png,0
4,00004.png,0


In [2]:
from skimage.io import imread

%time ic_train = [imread('00_input/train/' + name) for name in data_train.filename]

CPU times: user 11.1 s, sys: 1.07 s, total: 12.2 s
Wall time: 17.1 s


In [3]:
# %time ic_train = [resize(img, (32,32)) for img in ic_train]

In [4]:
from skimage.color import rgb2gray

def brightnees_channel(rgb):
    rgb = rgb.astype(np.float16)
    return (0.299 * rgb[:, :, 0] + 0.587 * rgb[:, :, 1] + 0.114 * rgb[:, :, 2]).astype(np.uint8)

# %time ic_train = [brightnees_channel(img) for img in ic_train]

In [5]:
import numpy as np
from skimage.transform import resize
from scipy.ndimage.filters import convolve

def _hog_normalize_block(block, method, eps=1e-5):
    if method == 'L1':
        out = block / (np.sum(np.abs(block)) + eps)
    elif method == 'L1-sqrt':
        out = np.sqrt(block / (np.sum(np.abs(block)) + eps))
    elif method == 'L2':
        out = block / np.sqrt(np.sum(block ** 2) + eps ** 2)
    elif method == 'L2-Hys':
        out = block / np.sqrt(np.sum(block ** 2) + eps ** 2)
        out = np.minimum(out, 0.2)
        out = out / np.sqrt(np.sum(block ** 2) + eps ** 2)
    else:
        raise ValueError('Selected block normalization method is invalid.')

    return out

def extract_hog(img, img_size=32, cell_size=8, block_size=2, block_stride=2, bins=8, eps=1e-8):
    cell_rows = cell_cols = cell_size
    block_row_cells = block_col_cells = block_size
    block_row_stride = block_col_stride = block_stride
    
    img = brightnees_channel(img)
    img = resize(img, (img_size,img_size))
    
    block_rows = block_row_cells * cell_rows
    block_cols = block_col_cells * cell_cols
    Dx = np.array([[-1, 0, 1]])
    Dy = np.array([[-1], [0], [1]])
    
    Ix = convolve(img, Dx, mode='constant')
    Iy = convolve(img, Dy, mode='constant')
    G = np.sqrt(Ix ** 2 + Iy ** 2)
    # theta
    T = np.arctan2(Iy, Ix)    
    T[T < 0] += np.pi
    hog = []
    
    for block_row_start in range(0, img.shape[0] - block_rows, block_row_stride):
        for block_col_start in range(0, img.shape[1] - block_cols, block_col_stride):

            G_block = G[block_row_start : block_row_start + block_rows,
                        block_col_start : block_col_start + block_cols]
            T_block = T[block_row_start : block_row_start + block_rows,
                        block_col_start : block_col_start + block_cols]
            v = []
            for block_row_cell in range(block_row_cells):
                for block_col_cell in range(block_col_cells):
                    G_cell = G_block[block_row_cell * cell_rows : block_row_cell * cell_rows + cell_rows,
                                     block_col_cell * cell_cols : block_col_cell * cell_cols + cell_cols]
                    T_cell = T_block[block_row_cell * cell_rows : block_row_cell * cell_rows + cell_rows,
                                     block_col_cell * cell_cols : block_col_cell * cell_cols + cell_cols]    
                    hist, _ = np.histogram(T_cell.flatten(), bins, range=(0, np.pi), weights=G_cell.flatten())
                    v.extend(hist)
            v = np.array(v)
            v = v / np.sqrt(np.sum(v ** 2) + eps)
            hog.extend(v)
            
    return np.array(hog)

def extract_hog_transform(imgs, cell_size=8, block_size=3, block_stride=2, bins=8, eps=1e-8):
#     return [extract_hog(i, cell_size, block_size, block_stride, bins, eps) for i in imgs]
    return Parallel(n_jobs=6)(delayed(extract_hog)(i, cell_size, block_size, block_stride, bins, eps) 
                                for i in imgs)

In [6]:
num_feat = extract_hog(ic_train[0]).shape[0]
print(num_feat)

2048


/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [7]:
%%time
from joblib import Parallel, delayed
from tqdm import tqdm_notebook as tqdm
    
X_train = Parallel(n_jobs=10)(delayed(extract_hog)(ic_train[i]) for i in tqdm(range(len(ic_train))))

/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 're


CPU times: user 7.7 s, sys: 1.19 s, total: 8.9 s
Wall time: 5min 25s


In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.cross_validation import cross_val_score

# kfolds = StratifiedKFold(5, random_state=17).split(X_train, y_train)

/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

parameters = {
    'C': [0.4, 0.6],
    'penalty': ['l2'], # 'l1'
#     'loss': 'squared_hinge', ), # 'hinge'
}
svc = LinearSVC()
grid = GridSearchCV(svc, parameters, scoring='accuracy',
                    cv=StratifiedKFold(5, random_state=17).split(X_train, y_train),
                    verbose=1, n_jobs=8)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:  4.2min remaining:  2.8min
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:  6.1min finished


GridSearchCV(cv=<generator object _BaseKFold.split at 0x7f12fc05d620>,
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=8,
       param_grid={'C': [0.4, 0.6], 'penalty': ['l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=1)

In [15]:
print(grid.best_score_)
print(grid.best_params_)

0.926266928511
{'C': 0.4, 'penalty': 'l2'}


In [13]:
print(grid.best_score_)
print(grid.best_params_)

0.927312606799
{'C': 0.2, 'penalty': 'l2'}


In [98]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline

extracter = FunctionTransformer(extract_hog_transform, validate=False)
svc = LinearSVC(C=0.1)
pipe = make_pipeline(extracter, svc)

In [102]:
from sklearn.cross_validation import cross_val_score

/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [119]:
for kw_args in [
#         {'cell_size': 6, 'block_size': 2, 'block_stride': 2, 'bins': 8}, 0.9436
        {'cell_size': 4, 'block_size': 2, 'block_stride': 2, 'bins': 8}, # 9516
        {'cell_size': 6, 'block_size': 2, 'block_stride': 1, 'bins': 8},
    ]:
    print(kw_args)
#     X_train = FunctionTransformer(extract_hog_transform, validate=False, kw_args=kw_args).transform(ic_train)  
    X_train = Parallel(n_jobs=12)(delayed(extract_hog)(ic_train[i], **kw_args) for i in tqdm(range(len(ic_train))))
    kfolds = StratifiedKFold(5, random_state=17).split(X_train, y_train)
    svc = LinearSVC(C=0.1)
    print(cross_val_score(svc, X_train, y_train, scoring='accuracy', cv=kfolds, verbose=1, n_jobs=-1).mean())

{'block_size': 2, 'bins': 8, 'block_stride': 2, 'cell_size': 4}


/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 're

589/|/  2%|| 589/39209 [00:21<23:06, 27.85it/s]

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.9min finished


0.951643710248
{'block_size': 2, 'bins': 8, 'block_stride': 1, 'cell_size': 6}


/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/ilivans/.virtualenvs/cmn3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 're

error: 'i' format requires -2147483648 <= number <= 2147483647

In [32]:
def fit_and_classify(clf, X_train, y_train, X_test):
    return clf.fit(X_train, y_train).predict(X_test)

%time y_pred = fit_and_classify(LinearSVC(), X_train, y_train, X_test)

CPU times: user 2min 45s, sys: 908 ms, total: 2min 46s
Wall time: 2min 45s


In [41]:
data_test = data_train.ix[:len(ic_test) - 1, :].copy()
data_test.iloc[:, 1] = y_pred
data_test.to_csv('test.csv', index=False)